In [1]:
import dspy
import dspy
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Your custom env var name
geminai_api_key= os.getenv("gemini")

if geminai_api_key :

    lm = dspy.LM("gemini/gemini-2.5-flash", api_key=geminai_api_key)
    dspy.configure(lm=lm)
else :
    print("GEMINI API KEY NOT FOUND")



In [2]:
calculator = dspy.ChainOfThought("question -> answer: float")
calculator(question= "what is 12.5 * 6.3 ?")

Prediction(
    reasoning='To find the answer, I need to multiply 12.5 by 6.3.12.5 * 6.3 = 78.75',
    answer=78.75
)

## initial working module  of **mind Reading AI**

In [3]:
from dspy import Signature, InputField, OutputField

In [7]:
class QuestionGenerator(dspy.Signature):
    """ Generate yes or no questions  in order to guess the celebrity name in users's mind . You can ask in general or directly guess the name if you think singla is enough. You should never ask the same question twice"""
    past_questions: list[str]= dspy.InputField(desc="list of past questions asked to the user")
    past_answers:list[bool]= dspy.InputField(desc="past answer")
    new_question: str= dspy.OutputField(desc="new question that can help narrow down the celebrity name")
    guess_made: bool= dspy.OutputField(desc="If the new_question is the celibrity name guess , Set to True , if it is still a general question set to False")


class Reflection(dspy.Signature):
    """ Provide Reflection on the guessing process"""
    correct_celebrity_name:str = dspy.InputField(desc="the celebrity name in user's mind")
    final_guessor_question:str= dspy.InputField(desc="the final guess or question LM made")
    past_questions: list[str]= dspy.InputField(desc="all past question asked")
    past_answers:list[bool]= dspy.InputField(desc="all past answer")

    reflection:str= dspy.OutputField(desc="reflection on the guessing process , including waht was done well and what can be improved")


def ask(prompt, valid_responses=("y" , "n")):
    while True :
        response= input(f"{prompt}) ({'/'.join(valid_responses)}): ").strip().lower() 

        if response in valid_responses:
            return response
        print(f"Please enter one of :{', '.join(valid_responses)}")
        


In [8]:
def ask(prompt, valid_responses=("y" , "n")):
    while True :
        response= input(f"{prompt}) ({'/'.join(valid_responses)}): ").strip().lower() 

        if response in valid_responses:
            return response
        print(f"Please enter one of :{', '.join(valid_responses)}")
        


In [15]:
class CelebrityGuesser(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator= dspy.ChainOfThought(QuestionGenerator)
        self.reflection= dspy.ChainOfThought(Reflection)
        self.max_tries= max_tries 

    def forward(self):
        celebrity_name= input("Please think ofa a celibrity name, once you are ready , type the name and press enter...")
        past_questions= [] 
        past_answers= [] 

        correct_guess= False 

        for i in range(self.max_tries):
            question=self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers    
                
            )

            answer= ask(f"{question.new_question}").lower()== "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made and answer:
                correct_guess= True 
                break 

        if correct_guess:
                print("yay i got it right  ")

        else :
                print("oops, I cou't guess it right")

        reflection_output= self.reflection(
                    correct_celebrity_name=celebrity_name,
                    final_guessor_question= question.new_question,
                    past_question=past_questions,
                    past_answer=past_answers    
                )

        print(reflection_output)


            

        

In [16]:
celebrity_guess= CelebrityGuesser() 
celebrity_guess()

2025/09/07 01:28:48 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['correct_celebrity_name', 'final_guessor_question']. Missing: ['past_questions', 'past_answers'].


oops, I cou't guess it right
Prediction(
    reasoning='The final guess was "Is he known for playing the character Ant-Man?". Since the `correct_celebrity_name` field is empty, it indicates that the celebrity was not correctly identified. The model made a very specific guess without any prior information, as evidenced by the empty `past_questions` and `past_answers` lists. This approach is highly unlikely to succeed.',
    reflection='**What was done well:**\nGiven that no prior questions or answers were provided, the model had no information to work with. Therefore, it\'s difficult to identify anything that was done well in terms of a guessing strategy.\n\n**What can be improved:**\nThe primary area for improvement is the guessing strategy. The model jumped directly to a highly specific guess ("Is he known for playing the character Ant-Man?") without asking any preliminary, broad questions to gather information. A successful guessing strategy involves starting with general questions (

## save and load

In [18]:
celebrity_guess.save("celebrity.json", save_program=False)

In [19]:
celebrity_guess.load("celebrity.json")

In [20]:
celebrity_guess.save("celebrity/", save_program=True)

In [21]:
loaded = dspy.load("celebrity/")

In [22]:
loaded()

Please enter one of :y, n
Please enter one of :y, n


2025/09/07 01:33:38 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['correct_celebrity_name', 'final_guessor_question']. Missing: ['past_questions', 'past_answers'].


yay i got it right  
Prediction(
    reasoning='The `correct_celebrity_name` was not provided, so I cannot determine if the final guess "Robert Downey Jr." was correct. Therefore, I cannot provide specific reasoning for the accuracy or inaccuracy of this particular guess.',
    reflection='Without the `correct_celebrity_name`, `past_questions`, and `past_answers`, it is impossible to provide a meaningful reflection on the guessing process. I cannot assess what was done well (e.g., if the questions effectively narrowed down the possibilities, if the final guess was a logical conclusion based on the answers) or what could be improved (e.g., if questions were redundant, if a different line of questioning would have been more efficient, or if the guess was made prematurely). To offer a valuable reflection, I would need the complete context of the game.'
)
